# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
import nest_asyncio

nest_asyncio.apply()

# Set up Ollama for embeddings as well as LLM

In [2]:
# Set embedding model
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name='nomic-embed-text',
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

llm = Ollama(model='llama3.1', request_timeout=30.0)

# Configure Settings singleton
# Settings not needed - setting directly in the VectorStoreIndex call.
Settings.embed_model = ollama_embedding
Settings.llm = llm

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [3]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

ValueError: File metagpt.pdf does not exist.

In [4]:
from src.ingest_service import IngestService
# The Directory containing the knowledge documents used by the AI to do the analysis on the soil tests.
soil_knowledge_directory = r"G:\My Drive\Audios_To_Knowledge\knowledge\AskGrowBuddy\AskGrowBuddy\Knowledge\soil_test_knowlege"
# Load the documents

ingest_service = IngestService()
documents = ingest_service.load_obsidian_notes(soil_knowledge_directory)

ModuleNotFoundError: No module named 'src'

## Split into chunks. LlamaIndex likes the term nodes for chunks. I guess it sounds more Computer Sciencyish

In [ ]:
# chunk text
ingest_service = IngestService()
nodes = ingest_service.chunk_text(documents)

## Define Summary Index and Vector Index over the Same Data

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [ ]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

In [ ]:
print(len(response.source_nodes))

In [ ]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))